# Import

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 챕터5
## 역전파 이론

* 역전파: back propagation | 오차역전파법
    * 출력층에서 입력층 방향으로 (역방향) **오차를 전파**시키며 각 층의 **가중치를 업데이트**
    * 하나의 학습 데이터에 대한 비용함수의 그레디언트를 계산
    * 미분의 계산 비용과 정확도를 개선: **효율적, 적은 오차**

* 연쇄 법칙: Chain Rule
    * 여러 함수를 사슬처럼 연결하여 사용한다는 의미
    * 합성함수의 미분 = 구성 함수 각각을 미분한 후 곱한 것과 같음

* x에 대한 y의 미분
## $ \frac{dy}{dx} = \frac{dy}{dy} \frac{db}{da} \frac{da}{dx} $
## $ \frac{dy}{dx} = \frac{dy}{dy} \frac{dy}{db} \frac{db}{da} \frac{da}{dx} $

## 역전파 원리

* **합성함수의 미분은 구성 함수들의 미분의 곱으로 분해할 수 있다**
* 손실함수
    * 기계학습 모델의 성능을 측정하기 위해 사용하는 함수
    * 차이가 작을수록 성능이 좋음을 뜻함

## 계산 그래프로 살펴보기
* 순전파 계산 그래프
    * 통상적인 계산
* 역전파 계산 그래프
    * 미분값을 구하기 위한 계산
    * 순전파 시 이용한 데이터 사용
    * 순전파가 선행되어야함. 각 함수가 입력 변수의 값을 기억하여야 함

# 챕터 6 수동 역전파

## 6.1 Variable 클래스 추가 구현

In [ ]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        # grad = gradient: 기울기

## 6.2 Function 클래스 추가 구현

In [ ]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        #입력 변수 기억해둬야 역전파 가능
        self.input = input
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()

## 6.3 Square / Exp 클래스 추가 구현

In [ ]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

In [ ]:
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

## 6.4 역전파 구현

* 순전파 계산

In [ ]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))

In [ ]:
a = A(x)
b = B(a)
y = C(b)

* 역전파 계산

In [ ]:
y.grad = np.array(1.0)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)
print(x.grad)

3.297442541400256


# 챕터 7 역전파 자동화

## 역전파를 자동화
* 역전파 계산 코드를 수동으로 조합시 상당히 불편함
* 순전파를 한 번 수행하면 자동으로 역전파가 이루어지는 구조로 작성

## Define **by** run
* 동적 계산 그래프
* 딥러닝의 계산들을 **계산 시점**에 연결하는 방식

## 역전파 자동화하기

* 계산 그래프가 여러 개인 경우
    * 분기가 있거나 변수가 여러 번 사용되는 등 복잡한 경우
    * 복잡한 구조를 자동으로 역전파를 수행할 수 있는 구조를 마련

* 변수화 함수의 관계
    * 함수 입장에서의 변수: 입력, 출력
    * 변수 입장에서의 함수: 창조자

* **함수와 변수의 관계를 코드로 나타내기**
    * Variable 클래스에 변수 creator 추가, setter 추가할 것
    * [연결]을 동적으로 만드려는 기법
        * output 인스턴스에 set_creator(self) 를 추가하여 동적으로 연결
        * 연결된 Variable과 Fuction이 순전파된 그래프를 거슬러 올라갈 수 있게 연결함

* 계산 그래프를 거꾸로 거슬러 올라가는 코드
    * assert 문으로 조건을 충족하는지 확인: True가 아니면 예외 발생
    * 연결은 계산을 수행하는 시점에 발생 (순전파 시점)

* Linked list 데이터 구조
    * 노드들의 연결로 이루어진 데이터 구조

## 7.2 역전파 도전

1. 함수를 가져옴
2. 함수의 입력을 가져옴
3. 함수의 backward 메소드 호출

* Variable의 creator 생성

In [ ]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        f = self.creator
        if f is not None:
            x = f.input
            x.grad = f.backward(self.grad)
            x.backward()

In [ ]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self)
        self.input = input
        self.output = output
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()

In [ ]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

assert y.creator == C

In [ ]:
y.grad = np.array(1.0)
C = y.creator
b = C.input
b.grad = C.backward(y.grad)

B = b.creator
a = B.input
a.grad = B.backward(b.grad)

A = a.creator
x = A.input
x.grad = A.backward(a.grad)
print(x.grad)

3.297442541400256


## 반복문을 이용한 구현

In [ ]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)   # self.data와 self.grad의 타입을 맞춤

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)

class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self)
        self.input = input
        self.output = output
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()

class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

In [ ]:
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


# 챕터 9 파이썬 함수로 만들기

In [ ]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)} is noe surpported')
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)   # self.data와 self.grad의 타입을 맞춤

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)

class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self)
        self.input = input
        self.output = output
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()

class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

In [ ]:
def square(x):
    return Square()(x)
def exp(x):
    return Exp()(x)

In [ ]:
x = Variable(np.array(0.5))
y = square(exp(square(x)))

y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


# 챕터 10 테스트 프로그램

In [ ]:
import unittest

In [ ]:
class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)

    def test_backward(self):
        x = Variable(np.array(3.0))
        y = square(x)
        y.backward()
        expected = np.array(6.0)
        self.assertEqual(x.grad, expected)

# 이번 주 수업의 느낀 점 정리

* 딥러닝 레이어가 어떻게 동작하는지 알게 되었다
* 이를 통해 레이어 함수를 잘 설정하고, 은닉층 구성을 어떻게 진행해야 하는지를 생각해 보게 되었다
* 딥러닝 학습 과정에서의 역전파, 정확도가 올라가는 과정에 대해 알게 되었다
* 이론으로만 알고 있던 여러 함수들과 예제들을 직접 코드로 작성해 보며 실습할 수 있어서 좋았다